In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [ ]:
schedule = pd.read_csv('processed/schedule.csv', index_col=0)
states = pd.read_csv('processed/states.csv', index_col=0)

In [ ]:
df = pd.concat([states, schedule], axis=1)
df.corr()[schedule.columns]

In [ ]:
# Construct env prediction model using RF or GB
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_squared_log_error, mean_absolute_error, explained_variance_score, r2_score
import joblib
import autosklearn.regression

w = 6
h = 4
ncols = 4
nrows = math.ceil(states.shape[1]/ncols)
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(w*ncols, h*nrows))
fig.tight_layout(pad=6)

# build regression model for every column
for i, col in enumerate(states.columns):
    y = states[col].to_frame()
    y = y[y[col] != 'None']
    X =schedule.loc[y.index]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10000)
    scaler = MinMaxScaler().fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index) 

    scaler = MinMaxScaler().fit(y_train)
    y_train = pd.DataFrame(scaler.transform(y_train), columns=y_train.columns, index=y_train.index)
    y_test = pd.DataFrame(scaler.transform(y_test), columns=y_test.columns, index=y_test.index)

    y_train.to_csv('tmp/%s-train.csv' % col)
    y_test.to_csv('tmp/%s-test.csv' % col)

    # choose the best model from RF and GB
    # rf = RandomForestRegressor(random_state=1000)
    # gb = GradientBoostingRegressor(random_state=1000)
    # models = [gb, rf]
    # for model in models:
    #     # define the evaluation procedure
    #     cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1000)
    #     n_scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1)
    #     if np.mean(n_scores) > best_score:
    #         best_model = model
    #         best_score = np.mean(n_scores)
    #         best_model.fit(X=X_train, y=y_train.to_numpy().squeeze())
    #         joblib.dump(best_model, './models/%s.joblib' % col)
    #     print('%s: %.3f (%.3f)' % (col, np.mean(n_scores), np.std(n_scores)))
    
    # y_predict = best_model.predict(X_test)
    
    automl = autosklearn.regression.AutoSklearnRegressor(
        time_left_for_this_task=120,
        per_run_time_limit=30,
        tmp_folder='./autosklearn_regression_tmp',
        output_folder='./autosklearn_regression_out',
    )
    automl.fit(X_train, y_train, dataset_name=col)

    y_predict = automl.predict(X_test)
    r = int(i/ncols)
    c = int(i%ncols)
    ax[r][c].set_xlim(0, 1)
    ax[r][c].set_ylim(0, 1)
    ax[r][c].set_title(col)
    ax[r][c].set_xlabel('Observed Value')
    ax[r][c].set_ylabel('Predicted Value')
    ax[r][c].grid()
    ax[r][c].scatter(y_test, y_predict)

plt.savefig('results.png', dpi=300)